# CatBoost Model for TDE Classification

This notebook trains a CatBoost model with Optuna hyperparameter tuning.

**Key Features:**
- Uses the same cross-validation folds as all other models
- Handles class imbalance with `scale_pos_weight`
- Saves OOF and test predictions for ensemble
- Generates individual model submission

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import optuna
from sklearn.metrics import precision_recall_curve
import warnings

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
# Configuration
N_OPTUNA_TRIALS = 40  # CatBoost is slower, fewer trials
RANDOM_STATE = 15
MODEL_NAME = 'cat'

# Paths
DATA_DIR = os.path.join('..', 'data', 'processed')
MODEL_DIR = os.path.join('..', 'models')
SUBMISSION_DIR = os.path.join('..', 'submissions')
TRAIN_PATH = os.path.join(DATA_DIR, 'further_train_features.parquet')
TEST_PATH = os.path.join(DATA_DIR, 'further_test_features.parquet')
FOLDS_PATH = os.path.join(DATA_DIR, 'train_folds.csv')

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(SUBMISSION_DIR, exist_ok=True)

In [ ]:
# Load data
print("Loading data...")
train = pd.read_parquet(TRAIN_PATH)
test = pd.read_parquet(TEST_PATH)
folds = pd.read_csv(FOLDS_PATH)

# Merge folds with training data
train = train.merge(folds[['object_id', 'kfold']], on='object_id', how='left')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Class distribution: {train['target'].value_counts().to_dict()}")

In [ ]:
# Prepare features
drop_cols = ['object_id', 'target', 'split', 'SpecType', 'kfold']
feature_cols = [c for c in train.columns if c not in drop_cols]

X = train[feature_cols].copy()
y = train['target']
kfold = train['kfold']

X_test = test[feature_cols].copy()
object_ids_test = test['object_id']

# Calculate scale_pos_weight for imbalance
scale_pos_weight = (y == 0).sum() / (y == 1).sum()
print(f"Feature count: {len(feature_cols)}")
print(f"scale_pos_weight: {scale_pos_weight:.2f}")

In [ ]:
def objective(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15),
        'depth': trial.suggest_int('depth', 3, 6),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 10, 40),
        'eval_metric': 'PRAUC',
        'early_stopping_rounds': 50,
        'verbose': 0,
        'allow_writing_files': False,
        'random_seed': RANDOM_STATE
    }
    
    # Global OOF Evaluation: Initialize OOF predictions array
    oof_preds = np.zeros(len(y))
    
    for fold in range(5):
        train_idx = kfold != fold
        val_idx = kfold == fold
        
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        
        clf = CatBoostClassifier(**params)
        clf.fit(X_tr, y_tr, eval_set=(X_val, y_val))
        
        # Store predictions globally (do NOT calculate F1 here)
        preds = clf.predict_proba(X_val)[:, 1]
        oof_preds[val_idx] = preds
    
    # Calculate Global Metric (outside loop)
    # Optimize threshold on the full dataset
    prec, rec, thresholds = precision_recall_curve(y, oof_preds)
    f1_scores = 2 * (prec * rec) / (prec + rec + 1e-9)
    best_f1 = np.max(f1_scores)
    
    return best_f1

In [ ]:
# Run Optuna optimization
print(f"Running Optuna with {N_OPTUNA_TRIALS} trials...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_OPTUNA_TRIALS, show_progress_bar=True)

print(f"\nBest F1 Score: {study.best_value:.4f}")
print(f"Best params: {study.best_params}")

In [ ]:
# Train final model with best params
print("\nTraining final model with best params...")

best_params = study.best_params.copy()
best_params.update({
    'iterations': 3000,
    'eval_metric': 'PRAUC',
    'early_stopping_rounds': 150,
    'verbose': 0,
    'allow_writing_files': False,
    'random_seed': RANDOM_STATE
})

oof_preds = np.zeros(len(y))
test_preds = np.zeros(len(X_test))

for fold in range(5):
    train_idx = kfold != fold
    val_idx = kfold == fold
    
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    clf = CatBoostClassifier(**best_params)
    clf.fit(X_tr, y_tr, eval_set=(X_val, y_val))
    
    oof_preds[val_idx] = clf.predict_proba(X_val)[:, 1]
    test_preds += clf.predict_proba(X_test)[:, 1] / 5
    
    print(f"Fold {fold} complete.")

# Calculate final OOF F1
prec, rec, thresh = precision_recall_curve(y, oof_preds)
f1 = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-9)
best_thresh = thresh[np.argmax(f1)]
print(f"\nOOF F1 Score: {np.max(f1):.4f} at threshold {best_thresh:.4f}")

In [ ]:
# Save predictions
oof_df = pd.DataFrame({
    'object_id': train['object_id'],
    'target': y,
    f'pred_{MODEL_NAME}': oof_preds
})
oof_df.to_csv(os.path.join(MODEL_DIR, f'oof_{MODEL_NAME}.csv'), index=False)

test_df = pd.DataFrame({
    'object_id': object_ids_test,
    f'pred_{MODEL_NAME}': test_preds
})
test_df.to_csv(os.path.join(MODEL_DIR, f'preds_{MODEL_NAME}.csv'), index=False)

print(f"\nSaved OOF predictions to: models/oof_{MODEL_NAME}.csv")
print(f"Saved test predictions to: models/preds_{MODEL_NAME}.csv")

In [ ]:
# Create submission file for this model
# Apply optimal threshold from OOF
test_binary = (test_preds >= best_thresh).astype(int)

# Create submission dataframe
submission = pd.DataFrame({
    'object_id': object_ids_test,
    'target': test_binary
})

submission_path = os.path.join(SUBMISSION_DIR, f'submission_{MODEL_NAME}.csv')
submission.to_csv(submission_path, index=False)

print(f"\n=== {MODEL_NAME.upper()} Submission ===")
print(f"Threshold: {best_thresh:.4f}")
print(f"Prediction distribution: {np.bincount(test_binary)}")
print(f"Positive rate: {test_binary.mean():.4f}")
print(f"Saved to: {submission_path}")